In [1]:
%matplotlib inline
%pip install scipy
%pip install scikit-learn

import piplite
await piplite.install('scikit-learn')
import pandas as pd
import numpy as np

Dataset = pd.read_csv('diabetes.csv')

Dataset
# Dataset содержит 9 колонок: Pregnancies (Беременности), Glucose (Глюкоза), BloodPressure (Кровяное давление), SkinThickness (Толщина кожи), 
# Insulin (Инсулин), BMI (Индекс массы тела), DiabetesPedigreeFunction (Родословная диабета), Age (Возраст), 
# Outcome (Эта колонка отвечает за то подтвердился ли диагноз или нет). 

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.linear_model import LinearRegression
# Создали Pipeline. В узком смысле пайплайн – это модуль sklearn.pipeline, который позволяет автоматизировать предварительные преобразования данных перед обучением модели. 
# Добавили в него SimpleImputer. С помощью этого класса мы можем заменить значения NaN(отсутствующие значения) в наборе данных указанным заполнителем. 
# И добавили StandardScaler, который необходим для стандартизации значений данных в стандартный формат.

# Пайплайн
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaller', StandardScaler())
])

data = Dataset.drop('Outcome', axis = 1)
outcome = Dataset.Outcome

data_prepared = pipeline.fit_transform(data)

In [3]:
# Линейная регрессия
model = LinearRegression().fit(data_prepared,outcome)

some_data = data.iloc[:5]
some_output = outcome.iloc[:5]
data_prepared_some = pipeline.transform(some_data)

# Обучили модель с помощью алгоритма «Линейной регрессии». Сравнили первые 5 объектов. Вывели прогнозы и метки для них. 
print("Линейная регрессия. Прогнозы:", model.predict(data_prepared_some))
print("Линейная регрессия. Метки:",list(some_output))
# Если округлять полученные значения, то можно сделать вывод, что они совпадают с метками. 

Линейная регрессия. Прогнозы: [ 0.65175729  0.00573265  0.73642449 -0.0219232   0.83318937]
Линейная регрессия. Метки: [1, 0, 1, 0, 1]


In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

data_prepared = pipeline.transform(data)
data_predictions = model.predict(data_prepared)
linear_mse = mean_squared_error(outcome, data_predictions)
lin_rmse = np.sqrt(linear_mse)
print("Среднеквадратическое отклонение. Линейная регрессия:",lin_rmse)

Среднеквадратическое отклонение. Линейная регрессия: 0.39785855691820504


In [6]:
from sklearn.metrics import classification_report, confusion_matrix 
# Матрица ошибок – это показатель успешности классификации, где классов два или более. 
# Это таблица с 4 различными комбинациями сочетаний прогнозируемых и фактических значений.
# В таблице содержится информация сколько раз система приняла верное и сколько раз неверное решение по документам заданного класса. А именно:
# TP — истино-положительное решение;
# TN — истино-отрицательное решение;
# FP — ложно-положительное решение;
# FN — ложно-отрицательное решение
print(confusion_matrix(list(some_output),list(model.predict(data_prepared_some).round()))) 

# Точность системы (precision) в пределах класса – это доля документов действительно принадлежащих данному классу 
# относительно всех документов которые система отнесла к этому классу. 
# Полнота системы (recall) – это доля найденных классфикатором документов принадлежащих классу относительно всех документов этого класса в тестовой выборке.

# F-мера: Понятно что чем выше точность и полнота, тем лучше. 
# Но в реальной жизни максимальная точность и полнота не достижимы одновременно и приходится искать некий баланс.
# Поэтому, хотелось бы иметь некую метрику которая объединяла бы в себе информацию о точности и полноте нашего алгоритма. 
# Именно такой метрикой является F-мера.

# Accuracy — это показатель, который описывает общую точность предсказания модели по всем классам. 
# Это особенно полезно, когда каждый класс одинаково важен. 
# Он рассчитывается как отношение количества правильных прогнозов к их общему количеству.
print(classification_report(list(some_output),list(model.predict(data_prepared_some).round())))

[[2 0]
 [0 3]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         3

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [7]:
linearRegressor = LinearRegression()
linearRegressor.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': 'deprecated',
 'positive': False}

In [8]:
from sklearn.model_selection import GridSearchCV
# Гиперпараметры в линейной регрессии
param_grid = [
    {
        'copy_X':[False], 'fit_intercept':[False],
    }, 
    {
        'positive':[True],'n_jobs':[3,10,30]
    }
]
grid_search = GridSearchCV(linearRegressor,param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(data_prepared,outcome)
grid_search.best_params_

{'n_jobs': 3, 'positive': True}

In [9]:
import pickle
# Сохранили модель в формате pickle
file = open('models/linear_regressor.pkl', 'wb')
pickle.dump(grid_search.best_params_, file, pickle.HIGHEST_PROTOCOL)
file.close()

In [10]:
# Дерево решений
tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(data_prepared,outcome)

data_predictions = tree_regressor.predict(data_prepared)
linear_mse = mean_squared_error(outcome, data_predictions)
lin_rmse = np.sqrt(linear_mse)
print("Среднеквадратическое отклонение. Дерево решений:",lin_rmse)

Среднеквадратическое отклонение. Дерево решений: 0.0


In [11]:
tree_regressor.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [12]:
# Гиперпараметры в дереве решений.
param_grid = [
    {
        'max_features':[1,5,10], 'min_samples_leaf':[3,7,2]
    }, 
]
grid_search = GridSearchCV(tree_regressor,param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(data_prepared,outcome)
grid_search.best_params_

{'max_features': 5, 'min_samples_leaf': 7}

In [13]:
import pickle
# Сохранили модель в формате pickle

file = open('models/tree_regressor.pkl', 'wb')
pickle.dump(grid_search.best_params_, file, pickle.HIGHEST_PROTOCOL)
file.close()

In [5]:
# Разделяем данные для обучения и теста

from sklearn.model_selection import train_test_split 
data_train, data_test, outcome_train, outcome_test = train_test_split(data, outcome, test_size = 0.20)

In [14]:
# Метод опорных векторов
from sklearn.svm import SVC 
svclassifier = SVC(kernel='linear') 
svclassifier.fit(data_train, outcome_train)
y_pred = svclassifier.predict(data_test)

In [15]:
from sklearn.metrics import classification_report, confusion_matrix 
# Матрица ошибок в методе опорных векторов.
print(confusion_matrix(outcome_test,y_pred))

# Точность системы, полнота системы и F-мера в методе опорных векторов.
print(classification_report(outcome_test,y_pred))

[[82  8]
 [35 29]]
              precision    recall  f1-score   support

           0       0.70      0.91      0.79        90
           1       0.78      0.45      0.57        64

    accuracy                           0.72       154
   macro avg       0.74      0.68      0.68       154
weighted avg       0.74      0.72      0.70       154



In [16]:
svclassifier.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [17]:
from sklearn.model_selection import GridSearchCV
# Гиперпараметры в методе опорных векторов
param_grid = [
    {
        'C':[0.2], 'break_ties':[True], 'probability': [False]
    }, 
    {
        'C':[1.0], 'break_ties':[False], 'probability': [True]
    }
]
grid_search = GridSearchCV(svclassifier,param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(data_prepared,outcome)
grid_search.best_params_

{'C': 1.0, 'break_ties': False, 'probability': True}

In [18]:
import pickle
# Сохранили модель в формате pickle

file = open('models/svc.pkl', 'wb')
pickle.dump(grid_search.best_params_, file, pickle.HIGHEST_PROTOCOL)
file.close()

In [19]:
# Метод k-ближайших соседей
from sklearn.neighbors import KNeighborsClassifier 
knnclassifier = KNeighborsClassifier(n_neighbors=5) 
knnclassifier.fit(data_train, outcome_train)
y_pred = knnclassifier.predict(data_test)

In [20]:
from sklearn.metrics import classification_report, confusion_matrix 
# Матрица ошибок в методе k-ближайших соседей.
print(confusion_matrix(outcome_test,y_pred)) 

# Точность системы, полнота системы и F-мера в методе k-ближайших соседей.
print(classification_report(outcome_test,y_pred))

[[81  9]
 [31 33]]
              precision    recall  f1-score   support

           0       0.72      0.90      0.80        90
           1       0.79      0.52      0.62        64

    accuracy                           0.74       154
   macro avg       0.75      0.71      0.71       154
weighted avg       0.75      0.74      0.73       154



In [21]:
knnclassifier.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [22]:
from sklearn.model_selection import GridSearchCV
# Гиперпараметры в методе k-ближайших соседей
param_grid = [
    {
        'n_neighbors':[5,7,15], 'weights':['distance']
    }, 
    {
        'n_neighbors':[1,3,20], 'weights':['uniform']
    }
]
grid_search = GridSearchCV(knnclassifier,param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(data_prepared,outcome)
grid_search.best_params_

{'n_neighbors': 15, 'weights': 'distance'}

In [23]:
import pickle
# Сохранили модель в формате pickle

file = open('models/knn.pkl', 'wb')
pickle.dump(grid_search.best_params_, file, pickle.HIGHEST_PROTOCOL)
file.close()

In [24]:
# Случайный лес

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Оценки:",scores)
    print("Среднее:", scores.mean())
    print("Стандартное отклонение:",scores.std())

forest_model = RandomForestRegressor()
forest_model.fit(data_prepared,outcome)
forest_rmse = mean_squared_error(outcome, forest_model.predict(data_prepared))
print(np.sqrt(forest_rmse))
forest_rmse_scores = cross_val_score(forest_model, data_prepared,outcome,scoring='neg_mean_squared_error')
display_scores(np.sqrt(-forest_rmse_scores))

0.15070969264892464
Оценки: [0.41517795 0.44709089 0.38847604 0.34799707 0.41324398]
Среднее: 0.40239718537889246
Стандартное отклонение: 0.032959667449650336


In [25]:
from sklearn.metrics import classification_report, confusion_matrix 

# Матрица ошибок в методе "Случайный лес".
print(confusion_matrix(list(some_output),list(forest_model.predict(data_prepared_some).round()))) 

# Точность системы, полнота системы и F-мера в методе "Случайный лес".
print(classification_report(list(some_output),list(forest_model.predict(data_prepared_some).round())))

[[2 0]
 [0 3]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         3

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [26]:
# Гиперпараметры случайного леса

from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'n_estimators':[3,10,30], 'max_features':[2,4,7]
    }, 
    {
        'bootstrap':[False],'n_estimators':[3,10,30],'max_features':[2,3,4]   
    }
]

regressor = RandomForestRegressor()
grid_search = GridSearchCV(regressor,param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(data_prepared,outcome)
grid_search.best_params_

{'max_features': 2, 'n_estimators': 30}

In [27]:
import pickle
# Сохранили модель в формате pickle

file = open('models/random_forest.pkl', 'wb')
pickle.dump(grid_search.best_params_, file, pickle.HIGHEST_PROTOCOL)
file.close()

In [28]:
from sklearn.naive_bayes import GaussianNB

# Наивный байесовский классификатор
gnb_model = GaussianNB()
gnb_model.fit(data_prepared,outcome)
gnb_rmse = mean_squared_error(outcome, gnb_model.predict(data_prepared))
print(np.sqrt(gnb_rmse))
gnb_rmse_scores = cross_val_score(gnb_model, data_prepared,outcome,scoring='neg_mean_squared_error')
display_scores(np.sqrt(-gnb_rmse_scores))

0.48680506023116343
Оценки: [0.49674264 0.52223297 0.50323628 0.46442036 0.50487816]
Среднее: 0.4983020824505434
Стандартное отклонение: 0.018924487568488572


In [29]:
from sklearn.metrics import classification_report, confusion_matrix 

gnb_model.fit(data_train, outcome_train)
y_pred = svclassifier.predict(data_test)

# Матрица ошибок в наивном байесовском классификаторе.
print(confusion_matrix(outcome_test,y_pred)) 

# Точность системы, полнота системы и F-мера в наивном байесовском классификаторе.
print(classification_report(outcome_test,y_pred))

[[82  8]
 [35 29]]
              precision    recall  f1-score   support

           0       0.70      0.91      0.79        90
           1       0.78      0.45      0.57        64

    accuracy                           0.72       154
   macro avg       0.74      0.68      0.68       154
weighted avg       0.74      0.72      0.70       154



In [30]:
gnb_model.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [31]:
# Гиперпараметры в наивном байесовском классификаторе.
param_grid = [
    {
        'priors':[None], 'var_smoothing':[0.000000001, 0.0000001, 0.001]
    }, 
]

grid_search = GridSearchCV(gnb_model,param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(data_prepared,outcome)
grid_search.best_params_

{'priors': None, 'var_smoothing': 1e-09}

In [32]:
import pickle
# Сохранили модель в формате pickle

file = open('models/naive_bayes.pkl', 'wb')
pickle.dump(grid_search.best_params_, file, pickle.HIGHEST_PROTOCOL)
file.close()